In [67]:
import pandas as pd
import os
import re
import json
import requests
import datetime
from dateutil.relativedelta import *

In [246]:
# helper.py
def get_last_date_of_month(year, month):
    """Return the last date of the month.
    
    Args:
        year (int): Year, i.e. 2022
        month (int): Month, i.e. 1 for January

    Returns:
        date (datetime): Last date of the current month
    """
    today = datetime.datetime.now()
    if year == today.year and month == today.month:
        return datetime.datetime(year, month, today.day - 2)
    
    if month == 12:
        last_date = datetime.datetime(year, month, 31)
    else:
        last_date = datetime.datetime(year, month + 1, 1) + datetime.timedelta(days=-1)
    
    return last_date

def get_first_date_of_current_month(year, month):
    """Return the first date of the month.

    Args:
        year (int): Year
        month (int): Month

    Returns:
        date (datetime): First date of the current month
    """
    first_date = datetime.datetime(year, month, 1)
    return first_date

def get_symbol_for_isin(isin):
    url = 'https://query1.finance.yahoo.com/v1/finance/search'

    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.109 Safari/537.36',
    }

    params = dict(
        q=isin,
        quotesCount=1,
        newsCount=0,
        listsCount=0,
        quotesQueryId='tss_match_phrase_query'
    )

    resp = requests.get(url=url, headers=headers, params=params)
    data = resp.json()
    if 'quotes' in data and len(data['quotes']) > 0:
        return data['quotes'][0]['symbol']
    else:
        return None

In [185]:
# # Prices Dataloader
# import yfinance as yf
# data = yf.download(list(isin_ticker_dict.values()), '2018-01-01','2023-12-31',interval="1d")['Adj Close']
# data['Date'] = pd.to_datetime(data.index)
# data.set_index('Date', inplace=True)
# data = data.reindex(pd.date_range(data.index.min(), data.index.max())).sort_index(ascending=True).reset_index().rename(columns={'index': 'Date'})
# data.fillna(method = 'bfill', inplace = True)
# data.rename(columns = ticker_isin_dict, inplace = True)

# data.to_csv(r'C:\Users\mayan\OneDrive\Documents\PersonalFinance\MF_PRICES.csv', index = False)

[*********************100%***********************]  9 of 9 completed


In [70]:
MF_TRADEBOOK = r'C:\Users\mayan\OneDrive\Documents\PersonalFinance\MF_Tradebook'
STOCK_TRADEBOOK = r'C:\Users\mayan\OneDrive\Documents\PersonalFinance\Stock_Tradebook'
START_DATE = datetime.date(2018, 6, 1)
END_DATE = datetime.date.today()

TEST_DATE = datetime.date(2020, 6, 1)

In [186]:
# Loading MF historical data

mf_prices = pd.read_csv(r'C:\Users\mayan\OneDrive\Documents\PersonalFinance\MF_PRICES.csv')
df= pd.DataFrame()
for csv in os.listdir(MF_TRADEBOOK):
    df_temp = pd.read_csv(os.path.join(MF_TRADEBOOK, csv))
    df = pd.concat([df,df_temp], axis = 0)
df['symbol'] = df['symbol'].apply(lambda x: x.lower())
df = df.drop(labels = ['series', 'auction'], axis = 1)
df['trade_date'] = pd.to_datetime(df.trade_date)
df = df.sort_values('trade_date')

In [65]:
# storing symbol data assigned to unique ISIN
symbol_dict = {}
for i in df['isin'].unique():
    symbol_dict[i] = df[df['isin'] == i].iloc[0].symbol
    
isin_ticker_dict = {}
for isin in symbol_dict.keys():
    isin_ticker_dict[isin] = get_symbol_for_isin(isin)
    
ticker_isin_dict = dict([(value, key) for key, value in isin_ticker_dict.items()])

In [220]:
def get_net_for_month(year, month, df):
    first_date = get_first_date_of_current_month(year, month)
    last_date = get_last_date_of_month(year, month)
    df_seg = df[(df.trade_date.dt.date <= last_date.date()) & (df.trade_date.dt.date >= first_date.date())]
    
    return df_seg

In [221]:
get_net_for_month(2023, 6, df)

,symbol,isin,trade_date,exchange,segment,trade_type,quantity,price,trade_id,order_id,order_execution_time
94,uti nifty 50 index fund - direct plan,INF789F01XA0,2023-06-02,BSE,MF,buy,23.732,126.4037,726134541,726134541,2023-06-02T00:00:00
92,axis bluechip fund - direct plan,INF846K01DP8,2023-06-02,BSE,MF,buy,40.054,49.9300,726136730,726136730,2023-06-02T00:00:00
93,dsp mid cap fund - direct plan,INF740K01PX1,2023-06-02,BSE,MF,buy,20.281,98.6110,726124953,726124953,2023-06-02T00:00:00
95,edelweiss balanced advantage fund - direct plan,INF754K01BS2,2023-06-07,BSE,MF,buy,70.303,42.6700,732035991,732035991,2023-06-07T00:00:00
96,axis bluechip fund - direct plan,INF846K01DP8,2023-06-09,BSE,MF,buy,39.974,50.0300,733582958,733582958,2023-06-09T00:00:00
97,dsp mid cap fund - direct plan,INF740K01PX1,2023-06-09,BSE,MF,buy,20.157,99.2160,733583343,733583343,2023-06-09T00:00:00
98,uti nifty 50 index fund - direct plan,INF789F01XA0,2023-06-09,BSE,MF,buy,23.693,126.6115,733576089,733576089,2023-06-09T00:00:00
99,edelweiss balanced advantage fund - direct plan,INF754K01BS2,2023-06-14,BSE,MF,buy,70.188,42.7400,739221152,739221152,2023-06-14T00:00:00
102,uti nifty 50 index fund - direct plan,INF789F01XA0,2023-06-16,BSE,MF,buy,23.357,128.4321,742310970,742310970,2023-06-16T00:00:00
100,axis bluechip fund - direct plan,INF846K01DP8,2023-06-16,BSE,MF,buy,39.175,51.0500,742319262,742319262,2023-06-16T00:00:00


In [232]:
def evaluate_portfolio_for_fund(year, month, df, fund_isin):
    last_date = get_last_date_of_month(year, month)
    df_seg = df[(df.trade_date.dt.date <= last_date.date()) & (df["isin"] == fund_isin)]    
    df_seg['net_quantity'] = df_seg[['trade_type','quantity']].apply(lambda x: x[1] if x[0] == 'buy' else -x[0], axis = 1)
    
    quantity = sum(df_seg['net_quantity'])
    nav = mf_prices[mf_prices.Date == last_date][fund_isin].iloc[0]
    print(symbol_dict[fund_isin])
    print('Quantity: ', quantity)
    print('Close NAV: ', nav)
    return quantity*nav
    

In [251]:
evaluate_portfolio_for_fund(2023,7,df,'INF789F01XA0')

uti nifty index fund - direct plan
Quantity:  5636.366999999998
Close NAV:  134.83200073242188


C:\Users\mayan\AppData\Local\Temp\ipykernel_24796\577746791.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_seg['net_quantity'] = df_seg[['trade_type','quantity']].apply(lambda x: x[1] if x[0] == 'buy' else -x[0], axis = 1)


759962.6394721983

In [247]:
get_last_date_of_month(2023,7)

datetime.datetime(2023, 7, 18, 0, 0)

In [248]:
mf_prices[mf_prices.Date == get_last_date_of_month(2023,7)]['INF846K01DP8'].iloc[0]

52.90999984741211

In [193]:
mf_prices.Date = pd.to_datetime(mf_prices.Date)

In [194]:
mf_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2025 entries, 0 to 2024
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          2025 non-null   datetime64[ns]
 1   INF109K013N3  2025 non-null   float64       
 2   INF846K01DP8  2025 non-null   float64       
 3   INF789F01XA0  2025 non-null   float64       
 4   INF846K01EW2  2025 non-null   float64       
 5   INF209K01UN8  2025 non-null   float64       
 6   INF740K01PX1  2025 non-null   float64       
 7   INF754K01BS2  2025 non-null   float64       
 8   INF769K01DM9  2025 non-null   float64       
 9   INF247L01718  2025 non-null   float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 158.3 KB


In [200]:
fund_isin = 'INF247L01718'
mf_prices[mf_prices.Date == get_last_date_of_month(2020, 12)][fund_isin].iloc[0]

20.062700271606445

In [237]:
mf_prices.tail(5)

,Date,INF109K013N3,INF846K01DP8,INF789F01XA0,INF846K01EW2,INF209K01UN8,INF740K01PX1,INF754K01BS2,INF769K01DM9,INF247L01718
2020,2023-07-14,55.776699,52.75,133.576797,80.299698,48.279999,104.496002,43.990002,38.287998,26.531601
2021,2023-07-15,55.823002,52.93,134.575104,80.189903,48.599998,105.165001,44.209999,38.555000,26.507999
2022,2023-07-16,55.823002,52.93,134.575104,80.189903,48.599998,105.165001,44.209999,38.555000,26.507999
2023,2023-07-17,55.823002,52.93,134.575104,80.189903,48.599998,105.165001,44.209999,38.555000,26.507999
2024,2023-07-18,55.857601,52.91,134.832001,79.830498,48.599998,105.556000,44.240002,38.539001,26.648001


In [157]:
ticker_isin_dict

{'0P0000XVYC.BO': 'INF209K01UN8',
 '0P0000XVU7.BO': 'INF846K01EW2',
 '0P00017844.BO': 'INF769K01DM9',
 '0P0000XVTL.BO': 'INF846K01DP8',
 '0P0001F0CK.BO': 'INF247L01718',
 '0P0000XW2M.BO': 'INF740K01PX1',
 '0P0000XVU2.BO': 'INF789F01XA0',
 '0P0000XUYZ.BO': 'INF109K013N3',
 '0P0000XYE4.BO': 'INF754K01BS2'}

In [158]:
isin_ticker_dict

{'INF209K01UN8': '0P0000XVYC.BO',
 'INF846K01EW2': '0P0000XVU7.BO',
 'INF769K01DM9': '0P00017844.BO',
 'INF846K01DP8': '0P0000XVTL.BO',
 'INF247L01718': '0P0001F0CK.BO',
 'INF740K01PX1': '0P0000XW2M.BO',
 'INF789F01XA0': '0P0000XVU2.BO',
 'INF109K013N3': '0P0000XUYZ.BO',
 'INF754K01BS2': '0P0000XYE4.BO'}